In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm

In [4]:
!pip install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import open3d as o3d

In [6]:
!pip install pyntcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from pyntcloud import PyntCloud

In [8]:
!pip install pyntcloud plyfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import plyfile as ply

# 3 LiDARS 
(64 channels, ~100k points per scan, provided in binary PLY format):
- 1 top LiDARs (TOP), with a 360° view of the scene.
- 2 front-lateral LiDARs placed on the headlights with 270° of aperture, overlapping in the front of the ego vehicle. 

We combine the LiDAR pointclouds can be merged to achieve higher density in such region.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install pyvista

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
directory = '/content/drive/MyDrive/3DAR/LIDAR_FRONT_RIGHT/'

In [13]:
import os
import plyfile
from sklearn.cluster import DBSCAN
import numpy as np
import time

DBSCAN

In [10]:
# Create a dictionary to store the compressed point clouds for each class
compressed_clouds = {}

start = time.time()

counter = 0
for file in os.scandir(directory):
    if file.name.endswith('.ply') and file.is_file() and counter<100:
        plydata = plyfile.PlyData.read(file.path)
        points = np.column_stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']))
        labels = plydata['vertex']['ObjTag']
        points_labels = np.column_stack((points, labels))
        
        start = time.time()
        # Iterate over the classes
        for label in np.unique(labels):
            # Get the points for the current class
            class_points = points[labels == label]
            # Cluster the class points using DBSCAN
            dbscan = DBSCAN(eps=0.2, min_samples=10)
            dbscan.fit(class_points)
            # Get the representative points for each cluster
            cluster_labels = dbscan.labels_
            cluster_centers = np.array([class_points[cluster_labels == i].mean(axis=0) for i in np.unique(cluster_labels) if i != -1])
            compressed_clouds[file.name + "_" + str(label)] = cluster_centers
        end = time.time()
        print("Time taken for {}: {:.2f} seconds".format(file.name,end - start))
        counter += 1


end = time.time()
print("Time taken: {:.2f} seconds".format(end - start))

Time taken for Town01_Opt_ClearSunset_10112104586796580021.ply: 6.58 seconds
Time taken for Town01_Opt_ClearSunset_10107865452087165056.ply: 3.20 seconds
Time taken for Town01_Opt_ClearSunset_10026302636173738573.ply: 3.26 seconds
Time taken for Town01_Opt_ClearSunset_10045546190761286461.ply: 3.25 seconds
Time taken for Town01_Opt_ClearSunset_10136782375646021358.ply: 3.26 seconds
Time taken for Town01_Opt_ClearSunset_10042684887244231893.ply: 4.65 seconds
Time taken for Town01_Opt_ClearSunset_10141639728230508717.ply: 3.22 seconds
Time taken for Town01_Opt_ClearSunset_10048583854343856095.ply: 4.10 seconds
Time taken for Town01_Opt_ClearSunset_10143683330085033977.ply: 4.11 seconds
Time taken for Town01_Opt_ClearSunset_10187247851921022492.ply: 3.32 seconds
Time taken for Town01_Opt_ClearSunset_10352228426716885578.ply: 3.90 seconds
Time taken for Town01_Opt_ClearSunset_1022095815687329167.ply: 3.70 seconds
Time taken for Town01_Opt_ClearSunset_10239257090863668280.ply: 3.32 seconds


In [12]:
for key in compressed_clouds.keys():
    file_name, label = key.rsplit("_",1)
    compressed_cloud = compressed_clouds[key]
    original_points_count = len(points[labels == int(label)])
    compressed_points_count = len(compressed_cloud)
    if compressed_points_count > 0:
        compression_ratio = original_points_count / compressed_points_count
        print(f"Compression ratio for class {label} in file {file_name}: {compression_ratio}")
    else:
        print(f"No points in compressed cloud for class {label} in file {file_name}")

Compression ratio for class 0 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 0.0
No points in compressed cloud for class 1 in file Town01_Opt_ClearSunset_10112104586796580021.ply
Compression ratio for class 2 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 43.5
No points in compressed cloud for class 5 in file Town01_Opt_ClearSunset_10112104586796580021.ply
Compression ratio for class 7 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 950.0196078431372
Compression ratio for class 8 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 319.0833333333333
No points in compressed cloud for class 9 in file Town01_Opt_ClearSunset_10112104586796580021.ply
Compression ratio for class 10 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 5613.0
No points in compressed cloud for class 11 in file Town01_Opt_ClearSunset_10112104586796580021.ply
Compression ratio for class 17 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 0.0
Compression ratio for class

In [13]:
for label, compressed_cloud in compressed_clouds.items():
    original_points_count = len(points[labels == label])
    compressed_points_count = len(compressed_cloud)
    if compressed_points_count > 0:
        chamfer_distance = np.mean(np.min(np.linalg.norm(points[:, np.newaxis] - compressed_cloud, axis=2), axis=1))
        print("Chamfer distance for class {}: {:.2f}".format(label, chamfer_distance))
    else:
        print(f"No points in compressed cloud for class {label}")

Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_0: 11.81
No points in compressed cloud for class Town01_Opt_ClearSunset_10112104586796580021.ply_1
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_2: 14.24
No points in compressed cloud for class Town01_Opt_ClearSunset_10112104586796580021.ply_5


<ipython-input-13-74e7f226919d>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  original_points_count = len(points[labels == label])


Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_7: 3.42
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_8: 6.18
No points in compressed cloud for class Town01_Opt_ClearSunset_10112104586796580021.ply_9
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_10: 5.81
No points in compressed cloud for class Town01_Opt_ClearSunset_10112104586796580021.ply_11
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_17: 9.12
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_18: 15.35
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_40: 17.05
No points in compressed cloud for class Town01_Opt_ClearSunset_10112104586796580021.ply_41
Chamfer distance for class Town01_Opt_ClearSunset_10112104586796580021.ply_100: 4.52
No points in compressed cloud for class Town01_Opt_ClearSunset_10107865452087165056.ply_0
No points in compressed cloud for class Town01_Op

In [14]:
!pip install DracoPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import DracoPy

The following can be implemented for chamfer distance (misura quanto la ricostruzione del point cloud sia fedele all'originale) evaluation over each class, but it's extremely ram consuming

In [ ]:
# Start the timer
start = time.time()

# Initialize a counter to limit the number of point clouds processed
counter = 0

compressed_clouds = {}

# Iterate over the files in the directory
for file in os.scandir(directory):
    if file.name.endswith('.ply') and file.is_file() and counter<100:
        start_pc = time.time()
        # Read the ply file
        plydata = plyfile.PlyData.read(file.path)
        # Extract the XYZ coordinates of the points
        points = np.column_stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']))
        # Extract the labels of the points
        labels = plydata['vertex']['ObjTag']
        # Encode the point cloud
        encoded_data = DracoPy.encode(points)
        # Measure the size of the encoded data
        encoded_size = len(encoded_data)
        # Measure the size of the original data
        original_size = len(points)
        # Calculate the compression ratio
        compression_ratio = original_size/encoded_size
        print(f"Compression ratio for file {file.name}: {compression_ratio}")
        # Decode the point cloud
        decoded_points = DracoPy.decode(encoded_data)
        for label in np.unique(labels):
          # Get the points with the current label
          points_label = points[labels == int(label)]
          decoded_points_label = decoded_points.points[labels == int(label)]
          # Calculate the chamfer distance
          chamfer_distance = np.mean(np.min(pairwise_distances(points_label, decoded_points_label), axis=1))
          print("Chamfer distance for class {} in file {}: {:.2f}".format(label, file.name, chamfer_distance))
        # Write the encoded data to a file
        with open(file.path + '.drc', 'wb') as outfile:
            outfile.write(encoded_data)
        end_pc = time.time()
        print("Time taken for point cloud {}: {:.2f} seconds".format(file.name, end_pc - start_pc))
        counter += 1

# End the timer
end = time.time()

# Print the total time taken
print("Time taken for all point clouds: {:.2f} seconds".format(end - start))

Compression ratio for file Town01_Opt_ClearSunset_10112104586796580021.ply: 1.0723355396460894
Chamfer distance for class 0 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 9.39
Chamfer distance for class 1 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 56.21
Chamfer distance for class 2 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 29.04
Chamfer distance for class 5 in file Town01_Opt_ClearSunset_10112104586796580021.ply: 8.65


Pur senza metterlo dentro il loop, calcolare la distanza in questione uccide la ram. Anche se uso linalg. Il punto probabilmente è che con dbscan usavamo una specie di clustering. Qua ci sono troppi punti ed è computazionalmente troppo pesante

In [ ]:
# Start the timer
start = time.time()

# Initialize a counter to limit the number of point clouds processed
counter = 0

compressed_clouds = {}

# Iterate over the files in the directory
for file in os.scandir(directory):
    if file.name.endswith('.ply') and file.is_file() and counter<100:
        start_pc = time.time()
        # Read the ply file
        plydata = plyfile.PlyData.read(file.path)
        # Extract the XYZ coordinates of the points
        points = np.column_stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']))
        # Extract the labels of the points
        labels = plydata['vertex']['ObjTag']
        # Encode the point cloud
        encoded_data = DracoPy.encode(points)
        # Measure the size of the encoded data
        encoded_size = len(encoded_data)
        # Measure the size of the original data
        original_size = len(points)
        # Calculate the compression ratio
        compression_ratio = original_size/encoded_size
        print(f"Compression ratio for file {file.name}: {compression_ratio}")
        # Decode the point cloud
        decoded_points = DracoPy.decode(encoded_data)
        # Calculate the pairwise distances between the original point cloud and the decoded point cloud
        distances = np.linalg.norm(points[:, np.newaxis] - decoded_points.points, axis=-1)
        # Calculate the minimum distance for each point in the original point cloud
        min_distances = np.min(distances, axis=1)
        # Calculate the mean of the minimum distances
        chamfer_distance = np.mean(min_distances)
        print("Chamfer distance: {:.2f}".format(chamfer_distance))
        # Write the encoded data to a file
        with open(file.path + '.drc', 'wb') as outfile:
            outfile.write(encoded_data)
        end_pc = time.time()
        print("Time taken for point cloud {}: {:.2f} seconds".format(file.name, end_pc - start_pc))
        counter += 1

# End the timer
end = time.time()

# Print the total time taken
print("Time taken for all point clouds: {:.2f} seconds".format(end - start))

Compression ratio for file Town01_Opt_ClearSunset_10112104586796580021.ply: 1.0723355396460894


In [ ]:
for key in compressed_clouds.keys():
    file_name, label = key.rsplit("_",1)
    compressed_cloud = compressed_clouds[key]
    original_points_count = len(points[labels == int(label)])
    compressed_points_count = len(compressed_cloud)
    if compressed_points_count > 0:
        compression_ratio = original_points_count / compressed_points_count
        print(f"Compression ratio for class {label} in file {file_name}: {compression_ratio}")
    else:
        print(f"No points in compressed cloud for class {label} in file {file_name}")

In [40]:
for label, compressed_cloud in compressed_clouds.items():
    original_points_count = len(points[labels == label])
    compressed_points_count = len(compressed_cloud)
    if compressed_points_count > 0:
        chamfer_distance = np.mean(np.min(np.linalg.norm(points[:, np.newaxis] - compressed_cloud, axis=2), axis=1))
        print("Chamfer distance for class {}: {:.2f}".format(label, chamfer_distance))
    else:
        print(f"No points in compressed cloud for class {label}")